# VacationPy
----

#### Note
* Keep an eye on your API usage. Use https://developers.google.com/maps/reporting/gmp-reporting as reference for how to monitor your usage and billing.

* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [1]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os
import json

# Import API key
from api_keys import g_key

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [2]:
#load CSV
weather_data_load = "../output_data/cities.csv"

#create dataframe
weather_df = pd.read_csv(weather_data_load)

weather_df.head()

weather_df.dtypes


City            object
Country         object
Latitude       float64
Temperature    float64
Longitude      float64
Humidity       float64
Cloudiness     float64
Wind Speed     float64
dtype: object

### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [3]:
#configure gmaps
gmaps.configure(api_key=g_key)

#store latitude and longitudue
location = weather_df[["Latitude","Longitude"]]



#store humidity as weights
humidity = weather_df["Humidity"]

max_humid = humidity.max()

humidity = humidity.astype(float)

humidity.head()

0    70.0
1    73.0
2    87.0
3    87.0
4    40.0
Name: Humidity, dtype: float64

In [4]:
#create map

fig = gmaps.figure()

#create heatmap layer
heat_layer = gmaps.heatmap_layer(location,weights=humidity,max_intensity=max_humid)

#combine layer with map
fig.add_layer(heat_layer)


#print
fig


Figure(layout=FigureLayout(height='420px'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [5]:
#narrow city with criteria: max temp between 70 & 80 degrees, wind speed less than 10 MPH, and cloudiness = 0
narrow_city_df = weather_df.loc[(weather_df["Cloudiness"] == 0) & (weather_df["Wind Speed"]<=10) & 
                                (weather_df["Temperature"]>= 70) &(weather_df["Temperature"]<=80)]
#drop null values
narrow_city_df.dropna()

#reset index
narrow_city_df.reset_index(inplace=True)
del narrow_city_df['index']

narrow_city_df

,City,Country,Latitude,Temperature,Longitude,Humidity,Cloudiness,Wind Speed
0,East London,ZA,-33.0153,70.36,27.9116,85.0,0.0,2.86
1,Saint-Philippe,RE,-21.3585,78.80,55.7679,61.0,0.0,3.44
2,Monywa,MM,22.1167,73.54,95.1333,39.0,0.0,5.12
3,New Norfolk,AU,-42.7826,72.00,147.0587,38.0,0.0,1.01
4,Lakes Entrance,AU,-37.8811,72.00,147.9810,64.0,0.0,7.00
5,Pisco,PE,-13.7000,71.60,-76.2167,78.0,0.0,8.05
6,San Buenaventura,MX,27.0833,70.41,-101.5333,49.0,0.0,4.09
7,Cabrero,CL,-37.0333,71.60,-72.4000,68.0,0.0,3.44
8,Mattru,SL,7.6244,74.88,-11.8332,91.0,0.0,2.10
9,Lima,PE,-12.0432,75.00,-77.0282,78.0,0.0,9.22


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [6]:
#create hotel data frame
hotel_df = narrow_city_df.loc[:,["City","Country","Latitude","Longitude"]]


#add hotel name column
hotel_df["Hotel Name"]= ""

hotel_df


,City,Country,Latitude,Longitude,Hotel Name
0,East London,ZA,-33.0153,27.9116,
1,Saint-Philippe,RE,-21.3585,55.7679,
2,Monywa,MM,22.1167,95.1333,
3,New Norfolk,AU,-42.7826,147.0587,
4,Lakes Entrance,AU,-37.8811,147.9810,
5,Pisco,PE,-13.7000,-76.2167,
6,San Buenaventura,MX,27.0833,-101.5333,
7,Cabrero,CL,-37.0333,-72.4000,
8,Mattru,SL,7.6244,-11.8332,
9,Lima,PE,-12.0432,-77.0282,


In [7]:
#set base url
base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

#set parameters
params = {"type":"hotel",
         "keyword": "hotel",
         "radius": 5000,
         "key": g_key}


In [8]:
#run for loop
for index, row in hotel_df.iterrows():
    latitude = row["Latitude"]
    longitude = row["Longitude"]
    city_name = row["City"]
    
    #keyword to params dictionary
    params["location"]=f"{latitude},{longitude}"
    
    #combine url for API
    print(f"Retrieving Results for Index {index}:{city_name}.")
    response = requests.get(base_url,params=params).json()
    
    #get results
    results = response['results']
    
    #store hotel results in data frame
    try:
        print(f"Closest hotel in {city_name} is {results[0]['name']}.")
        hotel_df.loc[index,'Hotel Name'] = results[0]['name']
        
    except(KeyError, IndexError):
            print("Missing Field...skipping")

print("Done with Search")
    

Retrieving Results for Index 0:East London.
Closest hotel in East London is Blue Lagoon Hotel.
Retrieving Results for Index 1:Saint-Philippe.
Closest hotel in Saint-Philippe is Les Embruns Du Baril.
Retrieving Results for Index 2:Monywa.
Closest hotel in Monywa is Win Unity Hotel.
Retrieving Results for Index 3:New Norfolk.
Closest hotel in New Norfolk is The Woodbridge.
Retrieving Results for Index 4:Lakes Entrance.
Closest hotel in Lakes Entrance is The Esplanade Resort & Spa.
Retrieving Results for Index 5:Pisco.
Closest hotel in Pisco is HOSTAL MIRAMAR.
Retrieving Results for Index 6:San Buenaventura.
Closest hotel in San Buenaventura is Hotel Villa Jardín.
Retrieving Results for Index 7:Cabrero.
Closest hotel in Cabrero is Hotel San Martin.
Retrieving Results for Index 8:Mattru.
Missing Field...skipping
Retrieving Results for Index 9:Lima.
Closest hotel in Lima is Hotel Génova.
Retrieving Results for Index 10:Armidale.
Closest hotel in Armidale is Tattersalls Hotel Armidale.
Retri

In [10]:
hotel_df

,City,Country,Latitude,Longitude,Hotel Name
0,East London,ZA,-33.0153,27.9116,Blue Lagoon Hotel
1,Saint-Philippe,RE,-21.3585,55.7679,Les Embruns Du Baril
2,Monywa,MM,22.1167,95.1333,Win Unity Hotel
3,New Norfolk,AU,-42.7826,147.0587,The Woodbridge
4,Lakes Entrance,AU,-37.8811,147.9810,The Esplanade Resort & Spa
5,Pisco,PE,-13.7000,-76.2167,HOSTAL MIRAMAR
6,San Buenaventura,MX,27.0833,-101.5333,Hotel Villa Jardín
7,Cabrero,CL,-37.0333,-72.4000,Hotel San Martin
8,Mattru,SL,7.6244,-11.8332,
9,Lima,PE,-12.0432,-77.0282,Hotel Génova


In [11]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Latitude", "Longitude"]]

In [12]:
#establish markers and layers
markers = gmaps.marker_layer(locations,info_box_content=hotel_info)

#add layer to map
fig.add_layer(markers)

# Display figure
fig

Figure(layout=FigureLayout(height='420px'))